In [2]:
from minio import Minio

In [3]:
access_key = "minio"
password = "minio123"

In [4]:
client = Minio(endpoint='localhost:9000', access_key=access_key, secret_key=password, secure=False)

In [8]:
# if bucket not exist, create bucket
found = client.bucket_exists("files")
if not found:
    client.make_bucket("files")

In [38]:
import requests

In [43]:
r = requests.post("https://www.terorarananlar.pol.tr/ISAYWebPart/TArananlar/GetTerorleArananlarList")

In [45]:
data = r.json()

In [51]:
data

{'kirmizi': [{'ID': 0,
   'Adi': 'İlhami',
   'Soyadi': 'BALI',
   'DogumYeri': 'TABUK',
   'DogumTarihi': 1982,
   'Durum': 1,
   'GorselURL': ['/kurumlar/terorarananlar.pol.tr/Arsiv/22/FotoOrj_1.jpeg',
    '/kurumlar/terorarananlar.pol.tr/Arsiv/22/FotoOrj_2.jpeg'],
   'IlkGorselURL': '/kurumlar/terorarananlar.pol.tr/Arsiv/22/FotoOrj_1.jpeg',
   'AvatarURL': '[{"GorselURL":"\\/kurumlar\\/terorarananlar.pol.tr\\/Arsiv\\/22\\/FotoOrj_1.jpeg"},{"GorselURL":"\\/kurumlar\\/terorarananlar.pol.tr\\/Arsiv\\/22\\/FotoOrj_2.jpeg"}]',
   'TOrgutID': 0,
   'TOrgutAdi': 'DEAŞ',
   'TKategoriID': 6,
   'TKategoriAdi': 'kırmızı',
   'Sil': False},
  {'ID': 0,
   'Adi': 'Mustafa',
   'Soyadi': 'DOKUMACI',
   'DogumYeri': 'ADANA',
   'DogumTarihi': 1985,
   'Durum': 1,
   'GorselURL': ['/kurumlar/terorarananlar.pol.tr/Arsiv/28/FotoOrj_1.jpeg',
    '/kurumlar/terorarananlar.pol.tr/Arsiv/28/FotoOrj_2.jpeg'],
   'IlkGorselURL': '/kurumlar/terorarananlar.pol.tr/Arsiv/28/FotoOrj_1.jpeg',
   'AvatarURL': '[

In [ ]:
for key, value in data.items():
    for person in value:
        if person["GorselURL"] is None:
            print(person["Adi"], person["Soyadi"])
            continue
        for image in person["GorselURL"]:
            image_url = "https://www.terorarananlar.pol.tr" + image
            r = requests.get(image_url)
            if r.status_code != 200:
                print("Error: ", image_url)
                continue
            else:
                print(r.status_code)

In [102]:
import asyncio
import aiohttp
from time import perf_counter

start = perf_counter()

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status != 200:
            response.raise_for_status()
        return await response.json()

async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for key, value in data.items():
            for person in value:
                if person["GorselURL"] is None:
                    continue
                for image in person["GorselURL"]:
                    image_url = "https://www.terorarananlar.pol.tr" + image
                    tasks.append(fetch(session, image_url))
        # for url in urls:
            # tasks.append(fetch(session, url))
        return await asyncio.gather(*tasks)
    
async def main():
    urls = range(25000)
    htmls = await fetch_all(urls)
    print(htmls)

if __name__ == "__main__":
    asyncio.run(main())
    print(f"Time taken: {perf_counter() - start}")

Task exception was never retrieved
future: <Task finished name='Task-2655' coro=<download_image() done, defined at /tmp/ipykernel_34536/1704903339.py:4> exception=RuntimeError('Session is closed')>
Traceback (most recent call last):
  File "/tmp/ipykernel_34536/1704903339.py", line 5, in download_image
    async with session.get(url) as response:
  File "/home/kemal/.local/lib/python3.11/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
                 ^^^^^^^^^^^^^^^^
  File "/home/kemal/.local/lib/python3.11/site-packages/aiohttp/client.py", line 400, in _request
    raise RuntimeError("Session is closed")
RuntimeError: Session is closed
Task exception was never retrieved
future: <Task finished name='Task-2654' coro=<download_image() done, defined at /tmp/ipykernel_34536/1704903339.py:4> exception=RuntimeError('Session is closed')>
Traceback (most recent call last):
  File "/tmp/ipykernel_34536/1704903339.py", line 5, in download_image
    

In [94]:
picture = requests.get("https://www.terorarananlar.pol.tr/kurumlar/terorarananlar.pol.tr/Arsiv/22/FotoOrj_1.jpeg")

In [77]:
client.fput_object("files", "test.jpeg", "/home/kemal/Desktop/terordenarananlar/test.jpeg", content_type="image/jpeg")